## Analisis de audio en Python


## 1. Cargar audio estereo y trabajar con el

Cargaremos un audio estereo descargado del campues para primeramente mostrar sus caracteristicas principales solicitadas en el PDF y luego lo reproducideremos

### 1.1. Cargar el audio estereo y mostrar sus caracteristicas


In [ ]:
#Importamos las librerias necesarias
import librosa  
import os  
import sys

# Cargar el archivo de audio en estéreo
audio_path = "interstellar.mp3"
audio_stereo, sr = librosa.load(audio_path, sr=None, mono=False)

# Mostrar características básicas
print("Frecuencia de muestreo:", sr)  # Muestras por segundo
print("Forma del array de audio:", audio_stereo.shape)  # Mostraea (canales, muestras) si el audio que pasamos es estereo

# Calcular tamaño del archivo
file_size = os.path.getsize(audio_path) / (1024 * 1024)  # Convertir bytes a MB
print("Tamaño del archivo: {:.2f} MB".format(file_size))

### 1.2. Reproducir audio estereo

In [ ]:
from IPython.display import Audio

# Reproducimos nuestro audio:
Audio(audio_path)




## 2. Convertimos audio estereo a mono y trabajamos con el

Comvertiremos el audio estereo a mono y también mostraremos sus caracteristicas y lo reproduciremos como antes hemos hecho con el audio estereo

### 2.1. Convertimos audio estereo a mono y mostramos caracteristicas

In [ ]:

# Convertir a mono 
audio_mono = librosa.to_mono(audio_stereo)

# Mostrar características
print("Frecuencia de muestreo:", sr)
print("Forma del array mono:", audio_mono.shape)

# Tamaño estimado del archivo
size_mono = sys.getsizeof(audio_mono) / (1024 * 1024)
print("Tamaño en memoria aproximado: {:.2f} MB".format(size_mono))


### 2.2. Reproducimos el audio

In [ ]:
import soundfile as sf
import sys
import numpy as np

max_val = np.abs(audio_mono).max() # Calculamos el valor absoluto maximo para prevenir errores al guardar el archivo por si algun valor se sale de rango
if max_val > 1.0:
    audio_mono = audio_mono / max_val # Si hay valores fuera de rango dividimos todos los valores por el maximo
audio_mono = audio_mono.astype(np.float32) # Convertimos a float32, formato adecuado para archivo .wav

# Guardar como archivo .wav
sf.write("interstellar.wav", audio_mono, sr)

# Reproducir con widget
Audio("interstellar.wav")


## 3. Mostramos grafica de audios

In [ ]:
%matplotlib inline

### 3.1. Mostramos grafica audio mono

In [ ]:
import matplotlib.pyplot as plt

muestras = sr * 5 # Calculamos cuantas muestras hay en 5 s
tiempo_mono = np.linspace(0, 5, muestras) #alineamos el eje x con el numero de muestras que vamos a graficas

plt.figure(figsize=(12, 4)) #Creamos una figura con dimensiones en pulgadas
plt.plot(tiempo_mono, audio_mono[:muestras], color='blue') #Dibujamos la forma de onda del audio mono
plt.title("Audio MONO - primeros 5 segundos")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.grid(True) # Aniadimos una rejilla para facilirar la lectura de la grafica
plt.tight_layout() # Ajustamos bien los margenes
plt.show()




### 3.2. Mostramos grafica audio estereo

In [ ]:
tiempo_stereo = np.linspace(0, 5, muestras)

plt.figure(figsize=(12, 5))
plt.plot(tiempo_stereo, audio_stereo[0][:muestras], label="Canal izquierdo", alpha=0.7) #Dibujamos la forma de onda del canal izquierdo
plt.plot(tiempo_stereo, audio_stereo[1][:muestras], label="Canal derecho", alpha=0.7) # Dibujamos la forma de onda del canal derecho
plt.title("Audio ESTÉREO - primeros 5 segundos")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## 4. Diferencia entre audio estereo y mono

La principal diferencia entre estos dos tipos de audio podemos observarlo principalmente en la grafica, como vemos en el audio mono solo tenemos un unico canal de sonido es decir el sonido se esucha por igual por ambos altavoces, mientras que ene le audio estereo tenemos dos canales, el canal izquierdo, y el canal derecho, uno para el oido izquierdo y otro para el oido derecho.